In [7]:
import os
import numpy as np
import nibabel as nb
import pandas as pd
from pathlib import Path
from surface_distance import metrics


root = "/Users/rosana.eljurdi/Datasets/LymphSeg_dataset/val"
nifty_path = Path(root, "input_nifty")
patients  = list(sorted(nifty_path.rglob('sub-*')))

pred_nifty = Path(root, 'pred_nifty')
pred_nifty.mkdir(parents=True, exist_ok=True)
gt_packed_nifty = Path(root, 'packed_gt')
gt_packed_nifty.mkdir(parents=True, exist_ok=True)
in_packed_nifty = Path(root, 'packed_in')
in_packed_nifty.mkdir(parents=True, exist_ok=True)





In [8]:
basenames = []
results = []
for p in patients: 
    basenames.append(p.stem.split('.nii')[0])
    npy_path = Path(root, "predictions/{}".format(p.stem.split('.nii')[0]))
    nifty = str(p)

    nft = nb.load(nifty)

    patient  = list(sorted(npy_path.rglob('*.npy')))

    # Predictions
    a = np.array([np.load(p)[0] for p in patient])
    print(a.shape)
    b = nb.Nifti1Image(np.swapaxes(a, 0,1), affine = nft.affine, header = nft.header)
    save_to  = Path(pred_nifty, '{}.nii.gz'.format(p.stem.split('.nii')[0]))
    nb.save(b , str(save_to))
    
    # Groundtruth:
    gt_basenames = [Path(*(b.parts[:6] + ("gt_npy",) + b.parts[8:])) for b in patient]
    c = np.array([np.load(p) for p in gt_basenames])
 
    d = nb.Nifti1Image(np.swapaxes(c, 0,1), affine = nft.affine, header = nft.header)
    save_to  = Path(gt_packed_nifty, '{}.nii.gz'.format(p.stem.split('.nii')[0]))
    nb.save(d, str(save_to))
    
    #images:
    
    in_basenames = [Path(*(b.parts[:6] + ("input_npy",) + b.parts[8:])) for b in patient]
    e = np.array([np.load(p) for p in in_basenames])
 
    f = nb.Nifti1Image(np.swapaxes(e, 0,1), affine = nft.affine, header = nft.header)
    save_to  = Path(in_packed_nifty, '{}.nii.gz'.format(p.stem.split('.nii')[0]))
    nb.save(f, str(save_to))
    
    
    sf = metrics.compute_surface_distances(a.astype(bool),
                                           c.astype(bool), [1,1,1])
    
    Dice_3D = metrics.compute_dice_coefficient(a.astype(bool), c.astype(bool))
    Hauss_3d = metrics.compute_robust_hausdorff(sf, 95)
    MASD_3D = metrics.compute_average_surface_distance(sf)
    
    metric = {'patient':p.stem.split('.nii')[0], 'dice':Dice_3D, "hauss": Hauss_3d, "MASD": np.mean(MASD_3D)  }
    
    results.append(metric)
    
    print(p.stem, Dice_3D, Hauss_3d, np.mean(MASD_3D))


df_Result = pd.DataFrame(data=results)
df_Result.to_csv(Path(root, 'metrics.csv'))

print(df_Result.describe())

print(df_Result['dice'].mean(), df_Result['dice'].std())
print(df_Result['hauss'].mean(), df_Result['hauss'].std())
    
    
    
    
    
    


(181, 184, 184)
sub-1070629.nii 0.8280528715199128 4.123105625617661 1.804112814742961
(181, 184, 184)
sub-1071101.nii 0.0 170.36725037400822 113.84026527196303
(181, 184, 184)
sub-1071155.nii 0.6022515734420707 7.3484692283495345 1.768476624286527
(181, 184, 184)
sub-1071273.nii 0.7779116975386957 7.0710678118654755 1.7388968439924253
(181, 184, 184)
sub-1071348.nii 0.8697453039276242 108.36050941187015 10.475180941213004
(181, 184, 184)
sub-1071897.nii 0.0 102.74726273726225 72.61071183604786
(181, 184, 184)
sub-1072359.nii 0.6225059856344772 142.1442928857856 26.170553821344072
(181, 184, 184)
sub-1072518.nii 0.6183775371247001 79.38513714795737 10.79947398979522
(181, 184, 184)
sub-1075391.nii 0.7652957038510878 58.39520528262573 8.730297256663716
(181, 184, 184)
sub-1075729.nii 0.18519140362659503 34.19064199455752 7.397507700828287
(181, 184, 184)


/Users/rosana.eljurdi/opt/anaconda3/envs/LymphSeg/lib/python3.10/site-packages/surface_distance/metrics.py:319: RuntimeWarning: invalid value encountered in double_scalars
  np.sum(distances_pred_to_gt * surfel_areas_pred) /


sub-1076353.nii 0.0 inf nan
(181, 184, 184)
sub-1078463.nii 0.938873640986362 2.0 0.38853992571467344
(181, 184, 184)
sub-1078624.nii 0.9092471270540221 3.3166247903554 0.7648960605713108
(181, 184, 184)
sub-1079452.nii 0.0 inf nan
            dice       hauss        MASD
count  14.000000   14.000000   12.000000
mean    0.508389         inf   21.374076
std     0.381257         NaN   35.422190
min     0.000000    2.000000    0.388540
25%     0.046298    7.140418    1.761082
50%     0.620442   68.890171    8.063902
75%     0.815518  133.698347   14.642244
max     0.938874         inf  113.840265
0.5083894889075391 0.3812565971434677
inf nan


In [40]:
path = Path( "/Users/rosana.eljurdi/Datasets/Lymphoma-defaced-brain-mni/derivatives/manual_segm/sub-1079452/ses-M000/anat/sub-1079452_ses-M000_dseg.nii.gz")
npy_file = nb.load(path)

print(np.unique(npy_file.get_fdata()))


    





[0.]
